### This notebook is about data wrangling for 1st capstone project, the goal is to predict airbnb listing price at Austin, TX. The dataset was taken from insideairbnb.com (http://insideairbnb.com/get-the-data.html)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) 

./111capstone1_data_wrangling.ipynb
./capstone1_data_story.ipynb
./capstone1_data_wrangling.ipynb
./listings.csv
./listings_clean.csv
./neighbourhoods.geojson
./Zipcodes.geojson
./.ipynb_checkpoints\capstone1_data_story-checkpoint.ipynb
./.ipynb_checkpoints\capstone1_data_wrangling-checkpoint.ipynb


In [2]:
listings_df = pd.read_csv('./listings.csv')
listings_df.shape

C:\Users\Hao\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(11250, 106)

#### So there are 11250 entries and 106 features in total. 
#### First, we will take a look at the features mostly are NA and drop them

In [3]:
listings_df.isnull().sum().sort_values(ascending = False).head(15)

neighbourhood_group_cleansed    11250
host_acceptance_rate            11250
thumbnail_url                   11250
medium_url                      11250
xl_picture_url                  11250
license                         11217
square_feet                     11046
monthly_price                   10473
weekly_price                    10318
notes                            5971
access                           4554
transit                          4233
host_about                       3932
interaction                      3769
neighborhood_overview            3714
dtype: int64

We can drop the features with > 10000 NA values, namely following features<br>

    neighbourhood_group_cleansed    11250
    host_acceptance_rate            11250
    thumbnail_url                   11250
    medium_url                      11250
    xl_picture_url                  11250
    license                         11217
    square_feet                     11046
    monthly_price                   10473
    weekly_price                    10318

In [4]:
listings_df.dropna(axis=1, thresh = 1000, inplace=True)
listings_df.shape

(11250, 97)

#### Next let's check if there is any feature that contains all the same values, we can discard it since it is not useful

In [5]:
feature_value_counts = [(i,len(listings_df[i].value_counts())) for i in listings_df.columns]
feature_value_counts.sort(key= lambda x:  x[1])
feature_value_counts[:15]

[('scrape_id', 1),
 ('last_scraped', 1),
 ('experiences_offered', 1),
 ('country_code', 1),
 ('country', 1),
 ('has_availability', 1),
 ('calendar_last_scraped', 1),
 ('requires_license', 1),
 ('jurisdiction_names', 1),
 ('is_business_travel_ready', 1),
 ('host_is_superhost', 2),
 ('host_has_profile_pic', 2),
 ('host_identity_verified', 2),
 ('market', 2),
 ('is_location_exact', 2)]

 So we can remove following features which contains same value across all rows

     ('scrape_id', 1),
     ('last_scraped', 1),
     ('experiences_offered', 1),
     ('country_code', 1),
     ('country', 1),
     ('has_availability', 1),
     ('calendar_last_scraped', 1),
     ('requires_license', 1),
     ('jurisdiction_names', 1),
     ('is_business_travel_ready', 1)

In [6]:
for i in listings_df.columns:
    if len(listings_df[i].value_counts()) == 1:
        listings_df.drop(i,inplace=True,axis=1)
listings_df.shape

(11250, 87)

#### Next, we will check what each column is about and decide which ones are related for our analysis (predict Airbnb listing price)

Let's list all features

In [7]:
[i for i in enumerate(listings_df.columns)]

[(0, 'id'),
 (1, 'listing_url'),
 (2, 'name'),
 (3, 'summary'),
 (4, 'space'),
 (5, 'description'),
 (6, 'neighborhood_overview'),
 (7, 'notes'),
 (8, 'transit'),
 (9, 'access'),
 (10, 'interaction'),
 (11, 'house_rules'),
 (12, 'picture_url'),
 (13, 'host_id'),
 (14, 'host_url'),
 (15, 'host_name'),
 (16, 'host_since'),
 (17, 'host_location'),
 (18, 'host_about'),
 (19, 'host_response_time'),
 (20, 'host_response_rate'),
 (21, 'host_is_superhost'),
 (22, 'host_thumbnail_url'),
 (23, 'host_picture_url'),
 (24, 'host_neighbourhood'),
 (25, 'host_listings_count'),
 (26, 'host_total_listings_count'),
 (27, 'host_verifications'),
 (28, 'host_has_profile_pic'),
 (29, 'host_identity_verified'),
 (30, 'street'),
 (31, 'neighbourhood'),
 (32, 'neighbourhood_cleansed'),
 (33, 'city'),
 (34, 'state'),
 (35, 'zipcode'),
 (36, 'market'),
 (37, 'smart_location'),
 (38, 'latitude'),
 (39, 'longitude'),
 (40, 'is_location_exact'),
 (41, 'property_type'),
 (42, 'room_type'),
 (43, 'accommodates'),
 (4

#### Features can be divided into subgroups
1) general features about the listing (0 to 12)

2) features about host (13 to 29)
    
3) features about property location (30 to 40)

4) features about property conditions (41 to 48)

5) features about property prices/stays (49 to 61)

6) features about property reviews (62 to 77)

7) other features (78 to end)

#### We will examine these features and decide which ones to keep

In [8]:
features_to_keep = []
listings_df.iloc[:4,:13]

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,picture_url
0,2265,https://www.airbnb.com/rooms/2265,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,This colorful and clean 1923 house was complet...,Zen East is situated in a vibrant & diverse mu...,NaN,A 2013 Genuine Buddy Scooter 125 may be availa...,5 min walk to Capitol Metro Rail (train that t...,"Several local restaurants, small clubs, music ...","Depending on your dates and arrival time, I am...",• Check-in time is 4 pm. Check out is 11 am. I...,https://a0.muscache.com/im/pictures/4740524/63...
1,5245,https://www.airbnb.com/rooms/5245,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,"This green, colorful, clean and cozy house was...",Situated in a vibrant & diverse multicultural ...,NaN,Please note: A two story studio was built in t...,NaN,NaN,"I should be available, upon your arrival, to a...",A brief profile for all guests along with phot...,https://a0.muscache.com/im/pictures/5167505/b3...
2,5456,https://www.airbnb.com/rooms/5456,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,Cute Private Studio apartment located in Willo...,Great central location for walking to Convent...,My neighborhood is ideally located if you want...,Parking on street requires a permit. Permits ...,"Bus stop around the block. Uber, Lyft, Ride, ...",Guests have access to patio.,I am happy to welcome my guests and show them in.,No Pets allowed. No smoking in the room. No m...,https://a0.muscache.com/im/pictures/14084884/b...
3,5769,https://www.airbnb.com/rooms/5769,NW Austin Room,NaN,Looking for a comfortable inexpensive room to ...,Looking for a comfortable inexpensive room to ...,Quiet neighborhood with lots of trees and good...,NaN,We are approximately 16 miles from downtown Au...,Gravel Parking Kitchen,We interact with our guests as little or as mu...,I will need to see identification at check in....,https://a0.muscache.com/im/pictures/23822033/a...


In [9]:
features_to_keep.extend(['id'])

We will only keep id, which uniquely identifies the listing property. Other features are either not useful (url) or requires NLP analysis and are out of scope for this analysis (name, summary, description)

In [10]:
listings_df.iloc[:4,13:30]

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,t,https://a0.muscache.com/im/users/2466/profile_...,https://a0.muscache.com/im/users/2466/profile_...,East Downtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t
1,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,t,https://a0.muscache.com/im/users/2466/profile_...,https://a0.muscache.com/im/users/2466/profile_...,East Downtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t
2,8028,https://www.airbnb.com/users/show/8028,Sylvia,2009-02-16,"Austin, Texas, United States",I am a licensed Real Estate Broker and owner o...,within a few hours,100%,t,https://a0.muscache.com/im/users/8028/profile_...,https://a0.muscache.com/im/users/8028/profile_...,East Downtown,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t
3,8186,https://www.airbnb.com/users/show/8186,Elizabeth,2009-02-19,"Austin, Texas, United States",We're easygoing professionals that enjoy meeti...,within an hour,100%,t,https://a0.muscache.com/im/users/8186/profile_...,https://a0.muscache.com/im/users/8186/profile_...,SW Williamson Co.,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t


In [11]:
features_to_keep.extend(['host_id','host_since','host_response_time','host_response_rate','host_is_superhost','host_verifications','host_identity_verified'])

We will keep host_id, host_since, response_time, host_response_rate, host_is_superhost, host_verifications.

In [12]:
listings_df.iloc[:4,30:41]

,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,latitude,longitude,is_location_exact
0,"Austin, TX, United States",East Downtown,78702,Austin,TX,78702,Austin,"Austin, TX",30.27750,-97.71398,f
1,"Austin, TX, United States",East Downtown,78702,Austin,TX,78702,Austin,"Austin, TX",30.27577,-97.71379,t
2,"Austin, TX, United States",East Downtown,78702,Austin,TX,78702,Austin,"Austin, TX",30.26112,-97.73448,t
3,"Austin, TX, United States",SW Williamson Co.,78729,Austin,TX,78729,Austin,"Austin, TX",30.45596,-97.78370,t


In [13]:
features_to_keep.extend(['zipcode','latitude','longitude'])

We will keep zipcode, latitude, longitude. Since this study is about Austin, TX, the city, state, market are not very useful. The location can be uniquely determined by latitude and longtitude as well as zipcode

In [14]:
listings_df.iloc[:4,41:49]

,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities
0,House,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."
1,House,Private room,2,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."
2,Guesthouse,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets live..."
3,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."


In [15]:
features_to_keep.extend(['property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities'])

We will keep all columns 

In [16]:
listings_df.iloc[:4,49:62]

,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm
0,$225.00,$500.00,$100.00,4,$30.00,30,90,30,30,90,90,30.0,90.0
1,$100.00,$500.00,$75.00,2,$35.00,30,60,30,30,60,60,30.0,60.0
2,$95.00,$100.00,NaN,2,$45.00,2,90,2,2,90,90,2.0,90.0
3,$40.00,NaN,NaN,2,$0.00,1,14,1,1,14,14,1.0,14.0


In [17]:
features_to_keep.extend(['price','security_deposit','cleaning_fee','guests_included'])

We will keep all price, guests_included

In [18]:
listings_df.iloc[:4,62:78]

,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,3 months ago,0,0,0,0,24,1,2009-03-17,2019-03-16,93.0,9.0,10.0,10.0,10.0,8.0,9.0
1,5 months ago,0,0,0,0,9,0,2009-03-19,2018-03-14,91.0,10.0,8.0,10.0,9.0,10.0,9.0
2,6 days ago,10,24,54,304,508,47,2009-03-08,2019-10-28,96.0,10.0,10.0,10.0,10.0,10.0,10.0
3,2 days ago,0,3,13,13,257,19,2010-04-10,2019-11-03,98.0,10.0,10.0,10.0,10.0,10.0,10.0


In [19]:
features_to_keep.extend(['number_of_reviews','availability_30','availability_60','availability_90','availability_365','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value'])

We will keep number_of_reviews and all review_scores

In [20]:
listings_df.iloc[:4,78:]

,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,f,strict_14_with_grace_period,f,f,3,2,1,0,0.18
1,f,strict_14_with_grace_period,f,f,3,2,1,0,0.07
2,f,strict_14_with_grace_period,f,t,1,1,0,0,3.90
3,f,moderate,t,t,1,0,1,0,2.20


In [21]:
features_to_keep.extend(['instant_bookable','cancellation_policy','reviews_per_month'])

We will keep instant_bookable, cancellation_policy and reviews_per_month

#### Now we start to drop features that are not useful

In [22]:
listings_df.shape # before drop

(11250, 87)

In [23]:
listings_df = listings_df[features_to_keep] # after drop
listings_df.shape

(11250, 38)

In [24]:
listings_df.drop_duplicates(subset = ['id'], inplace=True) # drop any duplicate 'id'
listings_df.shape  # so all 11250 rows have unique id, which means unique property

(11250, 38)

In [25]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11250 entries, 0 to 11249
Data columns (total 38 columns):
id                             11250 non-null int64
host_id                        11250 non-null int64
host_since                     11247 non-null object
host_response_time             7939 non-null object
host_response_rate             7939 non-null object
host_is_superhost              11247 non-null object
host_verifications             11250 non-null object
host_identity_verified         11247 non-null object
zipcode                        11157 non-null object
latitude                       11250 non-null float64
longitude                      11250 non-null float64
property_type                  11250 non-null object
room_type                      11250 non-null object
accommodates                   11250 non-null int64
bathrooms                      11235 non-null float64
bedrooms                       11245 non-null float64
beds                           11238 non-nul

#### We will  fill NA values

1 For price related, fill NA values with 0

2 For zipcode, fill NA values based on API query result using latitude and latitude

3 For other features, fill NA values with most common

In [26]:
listings_df['security_deposit'].fillna('$0',inplace=True)
listings_df['cleaning_fee'].fillna('$0',inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11250 entries, 0 to 11249
Data columns (total 38 columns):
id                             11250 non-null int64
host_id                        11250 non-null int64
host_since                     11247 non-null object
host_response_time             7939 non-null object
host_response_rate             7939 non-null object
host_is_superhost              11247 non-null object
host_verifications             11250 non-null object
host_identity_verified         11247 non-null object
zipcode                        11157 non-null object
latitude                       11250 non-null float64
longitude                      11250 non-null float64
property_type                  11250 non-null object
room_type                      11250 non-null object
accommodates                   11250 non-null int64
bathrooms                      11235 non-null float64
bedrooms                       11245 non-null float64
beds                           11238 non-nul

In [27]:
zipna = listings_df[listings_df['zipcode'].isna()].index
zipna = zipna.tolist()

print("Count of missing 'zipcode': %d\n" % len(zipna))
print(zipna)

Count of missing 'zipcode': 93

[72, 938, 941, 1028, 1041, 1112, 1355, 1378, 1487, 1608, 1612, 1794, 1802, 2177, 2185, 2354, 2356, 2362, 2400, 2434, 2477, 2490, 2562, 2568, 2635, 2789, 3046, 3129, 3146, 3170, 3253, 3261, 3299, 3575, 3667, 3858, 3940, 3954, 4115, 4222, 4233, 4237, 4332, 4527, 4905, 4909, 4910, 4915, 4916, 4920, 4930, 4932, 4933, 4942, 4946, 4947, 4949, 4952, 5234, 5581, 5770, 5830, 5850, 5896, 5954, 5999, 6175, 6183, 6281, 6385, 6493, 6667, 6725, 6760, 6762, 6840, 7093, 7242, 7345, 7392, 7473, 7477, 7497, 7506, 7510, 7518, 7519, 7537, 7551, 7594, 7706, 7857, 9949]


In [30]:
import requests
import re

API = r"https://www.mapquestapi.com/geocoding/v1/reverse?key="
API_KEY = "EwUEHDGEs4GnQz4baNOLSIAL9oKaGxog"

zipcodes = {}
for idx in zipna:
    lat = listings_df.loc[idx]["latitude"]
    lat = str(lat)
    
    long = listings_df.loc[idx]["longitude"]
    long = str(long)
    
    url = API + API_KEY + r"&location=" + lat + r"," + long + \
           r"&outFormat=json&thumbMaps=false"
    r = requests.get(url)

    API_zip = r.json()["results"][0]["locations"][0]["postalCode"]
    API_zip = re.findall("^\d*", API_zip)[0]
    
    zipcodes[idx] = API_zip

print(zipcodes)

{72: '78704', 938: '78746', 941: '78746', 1028: '78722', 1041: '78702', 1112: '78704', 1355: '78704', 1378: '78703', 1487: '78703', 1608: '78704', 1612: '78704', 1794: '78749', 1802: '78704', 2177: '78741', 2185: '78702', 2354: '78704', 2356: '78756', 2362: '78749', 2400: '78701', 2434: '78702', 2477: '78704', 2490: '78741', 2562: '78704', 2568: '78722', 2635: '78701', 2789: '78722', 3046: '78753', 3129: '78744', 3146: '78751', 3170: '78705', 3253: '78751', 3261: '78705', 3299: '78756', 3575: '78704', 3667: '78702', 3858: '78722', 3940: '78701', 3954: '78751', 4115: '78739', 4222: '78705', 4233: '78745', 4237: '78703', 4332: '78751', 4527: '78702', 4905: '78704', 4909: '78745', 4910: '78746', 4915: '78704', 4916: '78745', 4920: '78704', 4930: '78703', 4932: '78757', 4933: '78723', 4942: '78703', 4946: '78705', 4947: '78704', 4949: '78754', 4952: '78729', 5234: '78745', 5581: '78704', 5770: '78745', 5830: '78703', 5850: '78704', 5896: '78749', 5954: '78723', 5999: '78749', 6175: '78704'

In [41]:
for idx in zipna:
    listings_df.loc[idx, "zipcode"] = str(zipcodes.get(idx))
    
print("The number of NaNs for 'zipcode': %d" % \
      len(listings_df[listings_df.zipcode.isna()]))

The number of NaNs for 'zipcode': 0


In [88]:
listings_df['zipcode'] = listings_df['zipcode'].astype(str)
listings_df['zipcode'] = listings_df['zipcode'].apply(lambda x: re.sub(r'[a-zA-Z\s]','',x))
listings_df['zipcode'] = listings_df['zipcode'].apply(lambda x: re.sub(r'\.0','',x))
listings_df['zipcode'] = listings_df['zipcode'].astype(int)
aindex = listings_df[listings_df['zipcode'] ==78767].index
listings_df.loc[aindex,'zipcode'] = 78701
listings_df['zipcode'].value_counts().sort_index()

78613       1
78619       1
78620       2
78652       4
78653       1
78660       5
78669       2
78681       1
78701     680
78702    1467
78703     578
78704    2187
78705     612
78712       1
78717      52
78719       2
78721     233
78722     207
78723     352
78724      91
78725      72
78726      12
78727      91
78728      67
78729      91
78730      48
78731     132
78732      43
78733      72
78734     210
78735      65
78736      54
78737      99
78738      49
78739      23
78741     765
78742       7
78744     182
78745     534
78746     305
78747      47
78748     154
78749     117
78750      48
78751     480
78752     125
78753      87
78754     102
78756     155
78757     201
78758     214
78759     120
Name: zipcode, dtype: int64

In [89]:
nullcount = listings_df.isnull().sum()
nullcount = nullcount[nullcount>0]
nullcount.index
for col in nullcount.index:
    listings_df[col].fillna(listings_df[col].value_counts().idxmax(), inplace=True)

In [90]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11250 entries, 0 to 11249
Data columns (total 38 columns):
id                             11250 non-null int64
host_id                        11250 non-null int64
host_since                     11250 non-null object
host_response_time             11250 non-null object
host_response_rate             11250 non-null object
host_is_superhost              11250 non-null object
host_verifications             11250 non-null object
host_identity_verified         11250 non-null object
zipcode                        11250 non-null int32
latitude                       11250 non-null float64
longitude                      11250 non-null float64
property_type                  11250 non-null object
room_type                      11250 non-null object
accommodates                   11250 non-null int64
bathrooms                      11250 non-null float64
bedrooms                       11250 non-null float64
beds                           11250 non-nu

In [91]:
listings_df.to_csv('./listings_clean.csv',index=None)

### This is the end of data wrangling part. In this notebook, I have finished following steps:

1 Loaded the data into Pandas DataFrame

2 Removed features where majority is NaN value

3 Removed features where only has one unique value

4 Examined remaining features to evaluate whether it is related for modeling (predicting airbnb price), and removed those features considered as not useful

5 Filled NA values by most common value

6 Saved the dataframe to csv file for future analysis

After this processing, I will start to look into the DataFrame to prepare for a data story